In [1]:
# project 4 notebook b

In [2]:
#q8
!hdfs dfsadmin -fs hdfs://boss:9000 -report

Configured Capacity: 25821052928 (24.05 GB)
Present Capacity: 17506351957 (16.30 GB)
DFS Remaining: 17243529216 (16.06 GB)
DFS Used: 262822741 (250.65 MB)
DFS Used%: 1.50%
Replicated Blocks:
	Under replicated blocks: 167
	Blocks with corrupt replicas: 0
	Missing blocks: 85
	Missing blocks (with replication factor 1): 85
	Low redundancy blocks with highest priority to recover: 167
	Pending deletion blocks: 0
Erasure Coded Block Groups: 
	Low redundancy block groups: 0
	Block groups with corrupt internal blocks: 0
	Missing block groups: 0
	Low redundancy blocks with highest priority to recover: 0
	Pending deletion blocks: 0

-------------------------------------------------
Live datanodes (1):

Name: 172.18.0.4:9866 (p4-dn-1.p4_default)
Hostname: 18e00c58ba78
Decommission Status : Normal
Configured Capacity: 25821052928 (24.05 GB)
DFS Used: 262822741 (250.65 MB)
Non DFS Used: 8297923755 (7.73 GB)
DFS Remaining: 17243529216 (16.06 GB)
DFS Used%: 1.02%
DFS Remaining%: 66.78%
Configured Cac

In [3]:
import time
time.sleep(30)

In [4]:
#q9
import requests
file_status_req = requests.get("http://boss:9870/webhdfs/v1/single.csv?op=GETFILESTATUS")
file_length = file_status_req.json()["FileStatus"]["length"]
file_block_size = file_status_req.json()["FileStatus"]["blockSize"]

container_locations = {"lost": 0}
for offset in range(0, file_length, file_block_size):
    req = requests.get("http://boss:9870/webhdfs/v1/single.csv?op=OPEN&offset={}&noredirect=true".format(offset))
    if req.status_code == 403:
        container_locations["lost"] += 1 
    else :
        output_str = req.json()["Location"]
        container_loc = (output_str.split("/")[2].split(":")[0])
        if container_loc not in container_locations:
            container_locations[container_loc] = 1
        else:
            container_locations[container_loc] += 1

container_locations

{'lost': 85, '18e00c58ba78': 82}

In [5]:
#q10
count = 0
target = "Single Family"

for offset in range(0, file_length, file_block_size):
    req = requests.get("http://boss:9870/webhdfs/v1/single.csv?op=OPEN&offset={}&length={}".format(offset, file_block_size))
    if req.status_code != 403:
        new_line = True
        str_buffer = (req.content).decode("utf-8")
        lines = str_buffer.split("\n")
        for line in lines:
            if line.find(target) != -1:
                count += 1
count

218001